# Debugging GPU Memroy Usage in PyTorch

In [1]:
import sys
from pathlib import Path

# Make project root importable
ROOT = Path().resolve().parents[1]
sys.path.append(str(ROOT))

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from src.utils.variables.var_names import *
from src.utils.variables.coord_names import *
from src.data_processing.conversions.scalar_conversions import *
from src.config.env_loader import get_env_var
import src.learning.model_diagnostics as model_diagnostics
from src.learning.model_training import batch_data_by_num_stations, compute_val_loss

from src.data_processing.station_processor import ProcessStations
from src.data_processing.topography_processor import ProcessTopography
from src.data_processing.era5_processor import ProcessERA5

In [33]:
%autoreload 2
import deepsensor.torch
from deepsensor.train.train import train_epoch, set_gpu_default_device
from deepsensor.data.loader import TaskLoader
from deepsensor.data.processor import DataProcessor
from deepsensor.model.convnp import ConvNP
from deepsensor.data.utils import construct_x1x2_ds

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from mpl_toolkits.basemap import Basemap
import torch
from torch import optim
import os
import lab as B
from tqdm import tqdm
import cartopy.crs as ccrs
import cartopy.feature as cf

## Set up a Demo Dataset

In [6]:
get_env_var("CUDA_DEVICE")

'2'

In [7]:
torch.cuda.current_device()

0

In [8]:
# setup variables for experiment
var = TEMPERATURE
years = [2010, 2011, 2012, 2013, 2014]

train_years = [2010] #[2010, 2011, 2012, 2013]
validation_years = [2014]

# GPU settings
use_gpu = True
if use_gpu:
    cuda_device = int(get_env_var("CUDA_DEVICE"))
    set_gpu_default_device(backend="cuda", dev_id=2)

# visualisations of data
DEBUG_PLOTS = True

In [9]:
station_processor = ProcessStations()
topography_processor = ProcessTopography()
era5_processor = ProcessERA5()

In [10]:
topography_ds = topography_processor.load_ds(standardise_var_names=True, standardise_coord_names=True)
era5_ds = era5_processor.load_ds(mode="surface", years=years, standardise_var_names=True, standardise_coord_names=True)

In [11]:
era5_var = era5_processor.get_variable(era5_ds, var) # set variable to process - e.g. "temperature"
era5_var = kelvin_to_celsius(era5_var)
era5_ds[var] = era5_var

In [12]:
crop = False

crop_left = 166
crop_right = 176
crop_top = -38
crop_bottom = -48

In [13]:
if crop:
    era5_ds = era5_ds.sel(lat=slice(crop_top, crop_bottom), lon=slice(crop_left, crop_right))
    topography_ds = topography_ds.sel(lat=slice(crop_bottom, crop_top), lon=slice(crop_left, crop_right))

era5_ds_coarsen = era5_ds.coarsen(lat=5, lon=5, boundary='trim').mean()

In [14]:
ds_aux = topography_processor.compute_tpi(topography_ds, window_sizes=[0.1])

# coarsen the elevation data
ds_aux_coarse  = ds_aux.coarsen(lat=200, lon=200, boundary='trim').mean()

ds_aux = ds_aux.fillna(0)
ds_aux_coarse = ds_aux_coarse.fillna(0)

In [15]:
stations_df = station_processor.load_df(vars=[var], year_start=2010, year_end=2014)
stations_df.head()
stations_reset = stations_df.reset_index()
stations_reset.drop(columns=['station'], inplace=True)
stations_resample = stations_reset.groupby(['lat', 'lon']).resample("6h", on='time').mean()[['temperature']]
stations_resample = stations_resample.reset_index().set_index(['time', 'lat', 'lon']).sort_index()

if crop:
    stations_resample = stations_resample[(stations_resample.index.get_level_values('lat') > crop_bottom) & (stations_resample.index.get_level_values('lat') < crop_top) &
                                      (stations_resample.index.get_level_values('lon') > crop_left) & (stations_resample.index.get_level_values('lon') < crop_right)]

/home/crowelenn/niwa/convcnp-assim-nz/src/data_processing/station_processor.py:157: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'time' ('time',) The recommendation is to set join explicitly for this case.
  ds_comb = xr.concat([first, *station_iter], dim="station")
/tmp/ipykernel_96774/2471903628.py:5: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stations_resample = stations_reset.groupby(['lat', 'lon']).resample("6h", on='time').mean()[['temperature']]


In [16]:
era5_da = era5_ds.sel(lat=slice(ds_aux_coarse[LATITUDE].max(), ds_aux_coarse[LATITUDE].min()), lon=slice(ds_aux_coarse[LONGITUDE].min(), ds_aux_coarse[LONGITUDE].max()))

In [17]:
era5_ds_coarsen = era5_ds_coarsen[[var]]

In [18]:
data_processor = DataProcessor(x1_name=LATITUDE, x1_map=(era5_ds[LATITUDE].min(), era5_ds[LATITUDE].max()), x2_name=LONGITUDE, x2_map=(era5_ds[LONGITUDE].min(), era5_ds[LONGITUDE].max()))
era5_processed, station_processed = data_processor([era5_ds_coarsen, stations_resample])
ds_aux_processed, ds_aux_coarse_processed = data_processor([ds_aux, ds_aux_coarse], method='min_max')

x1x2_ds = construct_x1x2_ds(ds_aux_coarse_processed)
ds_aux_coarse_processed['x1_arr'] = x1x2_ds['x1_arr']
ds_aux_coarse_processed['x2_arr'] = x1x2_ds['x2_arr']

/home/crowelenn/niwa/convcnp-assim-nz/venv/src/deepsensor/deepsensor/data/processor.py:129: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  f"x1_map={x1_map} and x2_map={x2_map} have different ranges ({float(np.diff(x1_map))} "
/home/crowelenn/niwa/convcnp-assim-nz/venv/src/deepsensor/deepsensor/data/processor.py:130: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  f"and {float(np.diff(x2_map))}, respectively). "
/home/crowelenn/niwa/convcnp-assim-nz/venv/src/deepsensor/deepsensor/data/processor.py:128: UserWarning: x1_map=(-50.0, -32.0) and x2_map=(165.0, 180.0) have different ranges (18.0 and 15.0, respectively). This can lead to stretching

## Model Training

In [19]:
task_loader = TaskLoader(
        context = [station_processed, era5_processed, ds_aux_coarse_processed], 
        target = station_processed, 
        aux_at_targets = ds_aux_processed, 
        links = [(0, 0)])

### Investigating why the default internal grid is so dense

In [34]:
model = ConvNP(data_processor, task_loader, unet_channels=(64,)*5, likelihood="gnp")

dim_yc inferred from TaskLoader: (1, 1, 4)
dim_yt inferred from TaskLoader: 1
dim_aux_t inferred from TaskLoader: 2
Setting aux_t_mlp_layers: (64, 64, 64)
internal_density inferred from TaskLoader: 6877
encoder_scales inferred from TaskLoader: [7.270612185546023e-05, 0.0347222238779068, 0.007111109793186188]
decoder_scale inferred from TaskLoader: 0.00014541224371092046


In [41]:
import pandas as pd

from deepsensor.data.utils import (
    compute_xarray_data_resolution,
    compute_pandas_data_resolution,
)

In [46]:
variables = {
    "station_processed": station_processed,
    "era5_processed": era5_processed,
    "ds_aux_coarse_processed": ds_aux_coarse_processed,
}

for name, var in variables.items():
    if isinstance(var, (xr.DataArray, xr.Dataset)):
        # Gridded variable: use data resolution
        data_resolution = compute_xarray_data_resolution(var)
    elif isinstance(var, (pd.DataFrame, pd.Series)):
        # Point-based variable: calculate density
        data_resolution = compute_pandas_data_resolution(
            var, n_times=1000, percentile=5
        )
    else:
        continue

    print(f"{name}: data resolution = {1/data_resolution}")

station_processed: data resolution = 6877.478607561156
era5_processed: data resolution = 14.400000000000002
ds_aux_coarse_processed: data resolution = 70.3125130312425


The dataset with the highest internal density is 'station_processed' with an internal density of 6877. This is then selected as the ConvNP internal density.

Why is the internal density from the off-grid stations dataset so high?

From deepsensor: `# Point-based variable: calculate density based on pairwise distances between observations`

`The resolution is approximated as the Nth percentile of the distances
    between neighbouring observations, possibly using a subset of the dates in
    the data. The default is to use 1000 dates (or all dates if there are fewer
    than 1000) and to use the 5th percentile. This means that the resolution is
    the distance between the closest 5% of neighbouring observations.`

In [ ]:
# this is the function deepsensor use, but I modified it to also return the list of distances:
import scipy
def compute_pandas_data_resolution(
    df,
    n_times = 1000,
    percentile = 5,
):
    """Approximates the resolution of non-gridded pandas data with indexes time,
    x1, and x2.

    The resolution is approximated as the Nth percentile of the distances
    between neighbouring observations, possibly using a subset of the dates in
    the data. The default is to use 1000 dates (or all dates if there are fewer
    than 1000) and to use the 5th percentile. This means that the resolution is
    the distance between the closest 5% of neighbouring observations.

    Args:
        df (:class:`pandas.DataFrame` | :class:`pandas.Series`):
            Dataframe or series with indexes time, x1, and x2.
        n_times (int, optional):
            Number of dates to sample. Defaults to 1000. If "all", all dates
            are used.
        percentile (int, optional):
            Percentile of pairwise distances for computing the resolution.
            Defaults to 5.

    Returns:
        float: Resolution of the data (in spatial units, e.g. 0.1 degrees).
    """
    dates = df.index.get_level_values("time").unique()

    if n_times != "all" and len(dates) > n_times:
        rng = np.random.default_rng(42)
        dates = rng.choice(dates, size=n_times, replace=False)

    closest_distances = []
    df = df.reset_index().set_index("time")
    for time in dates:
        df_t = df.loc[[time]]
        X = df_t[["x1", "x2"]].values  # (N, 2) array of coordinates
        if X.shape[0] < 2:
            # Skip this time if there are fewer than 2 stationS
            continue
        X_unique = np.unique(X, axis=0)  # (N_unique, 2) array of unique coordinates

        pairwise_distances = scipy.spatial.distance.cdist(X_unique, X_unique)
        percentile_distances_without_self = np.ma.masked_equal(pairwise_distances, 0)

        # Compute the closest distance from each station to each other station
        closest_distances_t = np.min(percentile_distances_without_self, axis=1)
        closest_distances.extend(closest_distances_t)

    data_resolution = np.percentile(closest_distances, percentile)
    return data_resolution, closest_distances

In [56]:
res, dists = compute_pandas_data_resolution(station_processed)

In [ ]:
# this is the station resolution
print(res)

0.00014540212439201073


In [71]:
dists_sort = np.sort(dists)
dists_sort[:10]

array([0.0001454, 0.0001454, 0.0001454, 0.0001454, 0.0001454, 0.0001454,
       0.0001454, 0.0001454, 0.0001454, 0.0001454])

In [72]:
set(dists_sort)

{np.float64(0.00014540212439201073),
 np.float64(0.02263853027664162),
 np.float64(0.02470115107029566),
 np.float64(0.026429859896024165),
 np.float64(0.05185481681686933),
 np.float64(0.05546184612644111),
 np.float64(0.06499568582074207),
 np.float64(0.11388813290242192),
 np.float64(0.11610188959133654),
 np.float64(0.16085396807276103)}

There are only 10 different distance values. The 5th percentile will always return 0.000145 as this appears enough times to be the fifth percentile. If 0.02263 was the resolution instead, the internal density would be only 1/0.0226 ~ 44. This would be much more reasonable. There are two stations which are too close together and it is making the internal_density way too large.

In [77]:
stations_unique = (
    stations_resample
    .reset_index()[["lat", "lon"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

coords = stations_unique[["lat", "lon"]].to_numpy()

dist_matrix = scipy.spatial.distance.cdist(coords, coords, metric="euclidean")  # distances in degrees

dist_matrix_km = dist_matrix * 111.0

pairs = []

n = len(stations_unique)
for i in range(n):
    for j in range(i + 1, n):
        pairs.append((i, j, dist_matrix_km[i, j]))

distances = pd.DataFrame(
    pairs,
    columns=["station_i", "station_j", "distance_km"]
).sort_values("distance_km").reset_index(drop=True)

distances["lat_i"] = distances["station_i"].map(stations_unique["lat"])
distances["lon_i"] = distances["station_i"].map(stations_unique["lon"])
distances["lat_j"] = distances["station_j"].map(stations_unique["lat"])
distances["lon_j"] = distances["station_j"].map(stations_unique["lon"])

distances.head(20)



,station_i,station_j,distance_km,lat_i,lon_i,lat_j,lon_j
0,9,10,0.283363,-38.66100,177.98600,-38.65860,177.98513
1,7,8,44.799918,-38.97352,175.79080,-38.68400,176.07200
2,13,14,45.186606,-37.00884,174.80713,-36.60268,174.83458
3,10,11,47.158797,-38.65860,177.98513,-38.38228,178.30785
4,9,11,47.259449,-38.66100,177.98600,-38.38228,178.30785
5,7,12,100.389445,-38.97352,175.79080,-38.33174,175.15356
6,6,7,103.143419,-39.89320,175.65799,-38.97352,175.79080
7,8,12,109.188074,-38.68400,176.07200,-38.33174,175.15356
8,4,6,116.072688,-40.57728,176.44889,-39.89320,175.65799
9,6,8,141.870373,-39.89320,175.65799,-38.68400,176.07200


## Training Loop

In [21]:
task_loader.load_dask()

In [22]:
train_dates = era5_ds.sel(time=slice("2010-01-01", "2011-12-31")).time.values
val_dates = era5_ds.sel(time=slice("2012-01-01", "2012-06-30")).time.values

In [23]:
train_tasks = []
for date in tqdm(train_dates):
    task = task_loader(date, context_sampling=["split", "all", "all"], target_sampling=["split"], split_frac=0.5)
    train_tasks.append(task)


val_tasks = []
for date in tqdm(val_dates):
    task = task_loader(date, context_sampling=["split", "all", "all"], target_sampling=["split"], split_frac=0.5)
    val_tasks.append(task)


  0%|          | 0/2920 [00:00<?, ?it/s]

100%|██████████| 728/728 [00:15<00:00, 46.40it/s]


In [24]:
print(train_tasks[1])

time: 2010-01-01 06:00:00
ops: []
X_c: [(2, 7), ((1, 14), (1, 12)), ((1, 54), (1, 54))]
Y_c: [(1, 7), (1, 14, 12), (4, 54, 54)]
X_t: [(2, 4)]
Y_t: [(1, 4)]
Y_t_aux: (2, 4)



- three context sets
- X_c is the coordinates for the context sets
- Y_c is the values for the context sets
- 3, 20x20, 78x78 observations (Y_c)
- X_t is the target sensor coordinates
- Y_t is the target sensor values

In [25]:
task_batched = batch_data_by_num_stations(train_tasks, batch_size=16)

In [26]:
n_epochs = 3
train_losses = []
val_losses = []
lr=5e-5

output_model = False

val_loss_best = np.inf

opt = optim.Adam(model.model.parameters(), lr=lr)

for epoch in tqdm(range(n_epochs)):
    batch_losses = [train_epoch(model, task_batched[f'{num_stations}'], 
                                            batch_size=len(task_batched[f'{num_stations}']), 
                                            lr=lr, opt=opt) for num_stations in task_batched.keys()]
    
    train_loss = np.mean(batch_losses)
    train_losses.append(train_loss)

    with torch.no_grad():
        val_loss = compute_val_loss(model, val_tasks)
    val_losses.append(val_loss)

    if val_loss < val_loss_best:
        val_loss_best = val_loss
        if output_model:
            folder = os.path.join(get_env_var("OUTPUT_HOME"), "models", "downscaling", "temperature", "convcnp")
            if not os.path.exists(folder): os.makedirs(folder)
            torch.save(model.model.state_dict(), folder + f"model.pt")

    torch.cuda.empty_cache()

    print(f"Epoch {epoch} train_loss: {train_loss:.2f}, val_loss: {val_loss:.2f}")

  0%|          | 0/3 [00:00<?, ?it/s]/home/crowelenn/niwa/convcnp-assim-nz/venv/lib/python3.12/site-packages/lab/types.py:204: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numerictypes.
  for name in np.core.numerictypes.__all__ + ["bool"]:
 33%|███▎      | 1/3 [00:49<01:39, 49.95s/it]

Epoch 0 train_loss: 1.72, val_loss: 1.56


 67%|██████▋   | 2/3 [01:32<00:45, 45.53s/it]

Epoch 1 train_loss: 1.64, val_loss: 1.71


100%|██████████| 3/3 [02:14<00:00, 44.67s/it]

Epoch 2 train_loss: 1.61, val_loss: 1.78


tensor multiplication: [[16, 16000, 6], [16, 6, 8128]]

In [27]:
deepsensor.model.nps.compute_encoding_tensor(model, task).shape

(1, 9, 64, 64)